In [36]:
from pathlib import Path
import datetime as dt
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np

OUT_DIR = Path('../output')

In [37]:
WEEKDAYS_OPEN = ['Wednesday', 'Thursday', 'Sunday']
VOLUNTEERS_BY_WEEKDAY = {
    'Wednesday': ['__', '__'],
    'Thursday': ['Finlay', 'Sanna'],
    'Sunday': ['Alex M', '__'],
    }
MEETING_TIME = ('Sunday', 2, '13:30', '15:30')
#MEETING_TIME = ('Wednesday', 1, '17:30', '19:30')

def build_roster(year, month, weekdays_open=WEEKDAYS_OPEN, 
  volunteers_by_weekday=VOLUNTEERS_BY_WEEKDAY, meeting_time=MEETING_TIME):
    """
    INPUTS:
    
    - ``year``: integer
    - ``month``: integer 1--12
    
    
    OUTPUTS:
    
    A pandas data frame
    """
    d1 = dt.date(year, month, 1)
    d2 = d1 + relativedelta(months=1)
    ix = pd.date_range(d1, d2, closed='left')
    f = pd.DataFrame(index=ix)
    f['date'] = f.index.strftime('%Y-%m-%d')
    f['weekday'] = f.index.weekday_name
    
    # Assign volunteers
    f['volunteers'] = f['weekday'].map(lambda x:
      ' + '.join(volunteers_by_weekday.get(x, ["__", "__"])))
    
    # Restrict to given weekdays
    cond = f['weekday'].isin(weekdays_open)
    f = f[cond].copy()
    
    # Add notes, including staff meeting.
    f['notes'] = ''
    cond = f['weekday'] == meeting_time[0]
    cond &= f.index.day >= 7*(meeting_time[1] - 1)
    cond &= f.index.day <= 7*meeting_time[1]
    f.loc[cond, 'notes'] = 'Staff meeting {!s}--{!s}'.format(*meeting_time[2:])
    
    return f.reset_index(drop=True)

def get_year_month(roster, format="%Y%m"):
    """
    Given a roster, return the year and month it pertains to 
    in the given format.
    """
    return pd.to_datetime(roster['date']).dt.strftime(format).iat[0]

def to_markdown(df, path):
    """
    Save the given data frame as a Markdown table located
    at the given path.
    """
    path = Path(path)
    
    # Get column names
    cols = df.columns

    # Create a new DataFrame with just the markdown
    # strings
    df2 = pd.DataFrame([['---',]*len(cols)], columns=cols)

    #Create a new concatenated DataFrame
    df3 = pd.concat([df2, df])

    #Save as markdown
    df3.to_csv(str(path), sep="|", index=False)


In [38]:
roster = build_roster(2016, 12)
print(roster)

date = get_year_month(roster)
path = OUT_DIR/'roster_{!s}.md'.format(date)
to_markdown(roster, path)

          date    weekday      volunteers                       notes
0   2016-12-01   Thursday  Finlay + Sanna                            
1   2016-12-04     Sunday     Alex M + __                            
2   2016-12-07  Wednesday         __ + __                            
3   2016-12-08   Thursday  Finlay + Sanna                            
4   2016-12-11     Sunday     Alex M + __  Staff meeting 13:30--15:30
5   2016-12-14  Wednesday         __ + __                            
6   2016-12-15   Thursday  Finlay + Sanna                            
7   2016-12-18     Sunday     Alex M + __                            
8   2016-12-21  Wednesday         __ + __                            
9   2016-12-22   Thursday  Finlay + Sanna                            
10  2016-12-25     Sunday     Alex M + __                            
11  2016-12-28  Wednesday         __ + __                            
12  2016-12-29   Thursday  Finlay + Sanna                            
